In [2]:
PATH = "../data/전처리데이터/감자_filled_weekly_item.csv"
df = pd.read_csv(PATH)
df

,품목명,연도,주차,주차_일수,주간_평균단가,금액_합,반입량_합
0,감자(수입),2016,9,1,1313.500000,13135000.0,10000.0
1,감자(수입),2016,10,1,1295.000000,12950000.0,10000.0
2,감자(수입),2016,11,1,990.000000,9900000.0,10000.0
3,감자(수입),2016,12,2,939.198113,19356000.0,20600.0
4,감자(수입),2016,13,4,936.201020,30052000.0,30800.0
...,...,...,...,...,...,...,...
2335,홍감자,2025,23,6,1741.143229,38425500.0,24980.0
2336,홍감자,2025,24,6,1315.278940,57297500.0,44310.0
2337,홍감자,2025,25,6,1618.807914,105049000.0,65300.0
2338,홍감자,2025,26,6,1512.928202,52948500.0,36660.0


In [6]:
# FacetGrid 대응 개선: apply_korean_font 재정의
import numpy as _np
from matplotlib.axes import Axes as _Axes
from matplotlib import font_manager

# get_korean_fontprop, apply_korean_font_to_axes가 이미 정의되어 있다고 가정

def apply_korean_font(obj):
    prop = get_korean_fontprop()

    # axes 속성이 있으면 배열로 평탄화하여 각 Axes에 적용 (FacetGrid/Figure 등)
    if hasattr(obj, "axes"):
        axes_obj = getattr(obj, "axes")
        try:
            axes_list = list(_np.ravel(axes_obj))
        except Exception:
            axes_list = axes_obj if isinstance(axes_obj, (list, tuple)) else []
        if axes_list:
            for ax in axes_list:
                if ax is not None:
                    apply_korean_font_to_axes(ax, prop)
            # FacetGrid 전용 legend 처리
            if hasattr(obj, "_legend") and obj._legend is not None:
                obj._legend.set_title(obj._legend.get_title().get_text(), prop=prop)
                for txt in obj._legend.texts:
                    txt.set_fontproperties(prop)
            return

    # 단일 Axes 처리
    if isinstance(obj, _Axes):
        apply_korean_font_to_axes(obj, prop)
        return

    # 기타 객체는 무시
    return

print("apply_korean_font가 FacetGrid 대응으로 업데이트되었습니다.")


apply_korean_font가 FacetGrid 대응으로 업데이트되었습니다.


In [8]:
# 한글 폰트 강제 적용 유틸 + 플롯 함수 재정의(강제 폰트 적용)
import os
import matplotlib.pyplot as plt
from matplotlib import font_manager
import seaborn as sns
import pandas as pd

# 강제 폰트 로더
_def_font_paths = [
    r"C:\\Windows\\Fonts\\malgun.ttf",
    r"C:\\Windows\\Fonts\\malgunbd.ttf",
    r"C:\\Windows\\Fonts\\NanumGothic.ttf",
    r"C:\\Windows\\Fonts\\GULIM.TTC",
]

def get_korean_fontprop():
    for fp in _def_font_paths:
        if os.path.exists(fp):
            try:
                font_manager.fontManager.addfont(fp)
                return font_manager.FontProperties(fname=fp)
            except Exception:
                continue
    # 이름 기반 최후 시도
    for name in ["Malgun Gothic", "NanumGothic", "Gulim", "Batang"]:
        try:
            return font_manager.FontProperties(family=name)
        except Exception:
            pass
    return None


def apply_korean_font_to_axes(ax, prop):
    if prop is None:
        return
    # 제목/라벨
    ax.set_title(ax.get_title(), fontproperties=prop)
    ax.set_xlabel(ax.get_xlabel(), fontproperties=prop)
    ax.set_ylabel(ax.get_ylabel(), fontproperties=prop)
    # 눈금
    for lbl in ax.get_xticklabels() + ax.get_yticklabels():
        lbl.set_fontproperties(prop)
    # 범례
    leg = ax.get_legend()
    if leg is not None:
        leg.set_title(leg.get_title().get_text(), prop=prop)
        for txt in leg.get_texts():
            txt.set_fontproperties(prop)


def apply_korean_font(obj):
    prop = get_korean_fontprop()
    if hasattr(obj, "axes") and isinstance(getattr(obj, "axes"), (list, tuple)):
        # FacetGrid 등
        for ax in obj.axes:
            if ax is not None:
                apply_korean_font_to_axes(ax, prop)
        # FacetGrid legend
        if hasattr(obj, "_legend") and obj._legend is not None:
            obj._legend.set_title(obj._legend.get_title().get_text(), prop=prop)
            for txt in obj._legend.texts:
                txt.set_fontproperties(prop)
    else:
        # 단일 Axes
        apply_korean_font_to_axes(obj, prop)


# 기존 플롯 함수 재정의: 그린 뒤 한글 폰트 강제 적용

def plot_weekly_price_by_item(df: pd.DataFrame, item_name: str,
                              item_col: str = "품목명",
                              year_col: str = "연도",
                              week_col: str = "주차",
                              price_col: str = "주간_평균단가") -> None:
    required_cols = {item_col, year_col, week_col, price_col}
    missing_cols = [c for c in required_cols if c not in df.columns]
    if missing_cols:
        raise ValueError(f"DataFrame에 필요한 컬럼이 없습니다: {missing_cols}")

    data = (
        df.loc[df[item_col] == item_name, [item_col, year_col, week_col, price_col]]
          .dropna(subset=[year_col, week_col, price_col])
          .copy()
    )
    if data.empty:
        raise ValueError(f"'{item_name}' 데이터가 비어있습니다.")

    data[year_col] = pd.to_numeric(data[year_col], errors="coerce")
    data[week_col] = pd.to_numeric(data[week_col], errors="coerce")
    data[price_col] = pd.to_numeric(data[price_col], errors="coerce")
    data = data.dropna(subset=[year_col, week_col, price_col]).sort_values([year_col, week_col])

    plt.figure(figsize=(12, 5))
    ax = sns.lineplot(data=data, x=week_col, y=price_col, hue=year_col, marker="o")
    ax.set_title(f"{item_name} | 주차별 주간_평균단가 (연도별)")
    ax.set_xlabel("주차")
    ax.set_ylabel("주간_평균단가")
    ax.set_xlim(1, 53)
    ax.set_xticks(range(1, 54, 4))
    ax.legend(title="연도", bbox_to_anchor=(1.02, 1), loc="upper left")

    apply_korean_font(ax)
    plt.tight_layout()
    plt.show()


def facet_all_items(df: pd.DataFrame,
                    item_col: str = "품목명",
                    year_col: str = "연도",
                    week_col: str = "주차",
                    price_col: str = "주간_평균단가",
                    max_items: int = 6) -> None:
    required_cols = {item_col, year_col, week_col, price_col}
    missing_cols = [c for c in required_cols if c not in df.columns]
    if missing_cols:
        raise ValueError(f"DataFrame에 필요한 컬럼이 없습니다: {missing_cols}")

    base = df.dropna(subset=[item_col, year_col, week_col, price_col]).copy()
    base[year_col] = pd.to_numeric(base[year_col], errors="coerce")
    base[week_col] = pd.to_numeric(base[week_col], errors="coerce")
    base[price_col] = pd.to_numeric(base[price_col], errors="coerce")
    base = base.dropna(subset=[year_col, week_col, price_col])

    items = base[item_col].dropna().unique().tolist()[:max_items]
    if not items:
        raise ValueError("표시할 품목이 없습니다.")

    g = sns.FacetGrid(base[base[item_col].isin(items)], col=item_col, col_wrap=3, sharey=False, height=3.2)
    g.map_dataframe(sns.lineplot, x=week_col, y=price_col, hue=year_col, marker="o")
    g.add_legend(title="연도")
    g.set_axis_labels("주차", "주간_평균단가")
    g.set(xlim=(1, 53))
    for ax in g.axes.flat:
        ax.set_xticks(range(1, 54, 8))

    apply_korean_font(g)
    title_prop = get_korean_fontprop()
    g.figure.suptitle("품목명별 주차-주간_평균단가 (연도별)", y=1.02, fontproperties=title_prop)
    plt.tight_layout()
    plt.show()

print("한글 강제 적용 버전의 함수가 재정의되었습니다.")


한글 강제 적용 버전의 함수가 재정의되었습니다.
